This notebook demonstrates how to manually sync Gsheets data to the Popit database, as opposed to using the CLI.

In [ ]:
import requests
import gSheet_utils
import searchCLI
import membershipsUpdate_langs
import personsUpdate_langs

## Importing persons

Set ID and name of Gsheet to import, along with Popit base details:

In [ ]:
sheetID = 'gSheetID'
sheetName = 'MP_persons'

base_url = "http://api.popit.sinarproject.org"
token = open('../oAuth/token_ms.txt').read()
headers = {'Authorization': token.rstrip()}
sub_langs = ['ms']

Import the Gsheet as a pandas df, and col_AI_map, a dictionary containing the mapping of column names to GSheet AI notation. Store this in gSheet_details, which we will pass into our personsUpdate script.

In [ ]:
df, col_AI_map = gSheet_utils.importGSheetAsDF(sheetID, sheetName)
gSheet_details = {'sheetID':sheetID, 'sheetName': sheetName, 'col_AI_map': col_AI_map, 'sub_langs':sub_langs}

Finally, call the personsUpdate function, which updates persons details for the all supported language codes on the Popit database.

In [ ]:
personsUpdate_langs.personsUpdate(df, base_url, headers, gSheet_details)                

## Importing memberships

In [ ]:
sheetID = 'gSheetID'
sheetName = 'memberships_Democratic_Party_of_X'


#We can also change Popit details based on DB we're syncing to
base_url = "http://api.openhluttaw.org"
token = open('../oAuth/token_my.txt').read()
headers = {'Authorization': token.rstrip()}
sub_langs = ['my']

df, col_AI_map = gSheet_utils.importGSheetAsDF(sheetID, sheetName)
gSheet_details = {'sheetID':sheetID, 'sheetName': sheetName, 'col_AI_map': col_AI_map, 'sub_langs':sub_langs}

The memberships sheets follow a naming format of memberships_organisationName, where spaces in the organisation names are replaced with an underscore. 
Eg. memberships_XYZ_Corp or memberships_Democratic_Party_of_X.

We can get the org name as below,

In [18]:
orgName = ' '.join(sheetName.split('memberships_')[1].split('_'))
print(orgName)

Democratic Party of X


And search Popit to check if the organisation already exists in the DB.

In [ ]:
orgID = searchCLI.searchCLI(base_url, orgName, 'organizations', 'name', 'othernames')    

If the organisation does not exist, we import it in as a new organisation. We obtain the Popit ID for the organisation, which all the memberships in the spreadsheet will be tied to.

In [ ]:
if not orgID:
        orgP = {'name': orgName}
        url = base_url+ "/en/organizations/"
        r_en = requests.post(url, headers=headers, json=orgP)

        if r_en.ok:
            try:
                orgID = r_en.json()['result']['id']
            except KeyError:
                orgID = r_en.json()['id']
        else:
            print(orgP)
            print(r_en.content)

Finally, we call the memberships update function on each membership entry in our dataframe, passing in the orgID we obtained earlier.

In [ ]:
df.apply(lambda row: membershipsUpdate_langs.genPayload(base_url, headers, row, orgID, gSheet_details, sub_langs), axis=1)

## Syncing all sheets in a memberships spreadsheet

Depending on how you chose to structure your directories, you may have many memberships sheets contained to a single Google spreadsheet. We can run a loop to update all the sheets in one go without having to repeatedly specify the different sheetNames.

In [ ]:
#Get a list of all sheet names in the spreadsheet
sheetNames = gSheet_utils.getSheetNames(spreadsheetID)

In [ ]:
for sheetName in sheetNames:
    orgName = ' '.join(sheetName.split('memberships_')[1].split('_'))
    orgID = searchCLI.searchCLI(base_url, orgName, 'organizations', 'name', 'othernames', [])
    print("Importing for {}".format('orgName'))
    
    if not orgID: #if organisation doesn't exist in Popit
            print("Adding org for {}".format(orgName))
            orgP = {'name': orgName}
            url = base_url+ "/en/organizations/"
            r_en = requests.post(url, headers=headers, json=orgP)
            
            if r_en.ok:
                try:
                    orgID = r_en.json()['result']['id']
                except KeyError:
                    orgID = r_en.json()['id']
            else:
                print(orgP)
                print(r_en.content)
                
    df, col_AI_map = gSheet_utils.importGSheetAsDF(sheetID, sheetName)
    gSheet_details = {'sheetID':sheetID, 'sheetName': sheetName, 'col_AI_map': col_AI_map, 'sub_langs':sub_langs}
    df.apply(lambda row: membershipsUpdate_langs.genPayload(base_url, headers, row, orgID, gSheet_details, sub_langs), axis=1)
